In [10]:
from google.colab import drive
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
df_train = pd.read_csv('/content/drive/MyDrive/TP Proyecto/Entrega 3/Codigo E3/tiendas_caba_m2_train.csv')
df_val = pd.read_csv('/content/drive/MyDrive/TP Proyecto/Entrega 3/Codigo E3/tiendas_caba_m2_test.csv')

df_train = df_train.drop(columns=['customer_id'])
df_val = df_val.drop(columns=['customer_id'])

In [13]:
X_train = df_train.drop(columns=['cluster'])
y_train = df_train['cluster']

X_val = df_val.drop(columns=['cluster'])
y_val = df_val['cluster']

In [14]:
# Identificar columnas booleanas
boolean_cols = X_train.columns[X_train.nunique() == 2]
non_boolean_cols = X_train.columns.difference(boolean_cols)

# Escalar solo las columnas no booleanas
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy()

X_train_scaled[non_boolean_cols] = scaler.fit_transform(X_train[non_boolean_cols])
X_val_scaled[non_boolean_cols] = scaler.transform(X_val[non_boolean_cols])

In [15]:
# Definir el modelo SVM
svm_model = SVC()

param_dist = {
    'C': [0.01, 0.1, 1, 10, 100, 1000],  # Valores más amplios para C
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # Añadir el kernel sigmoid
    'degree': [2, 3, 4, 5],  # Mayor rango para el grado del kernel polinómico
    'gamma': ['scale', 'auto'] + [0.01, 0.1, 1, 10],  # Ampliar gamma
    'class_weight': [None, 'balanced']  # Balanceo de clases
}

# Usar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=svm_model,
    param_distributions=param_dist,
    n_iter=30,  # Aumentar el número de combinaciones para una búsqueda más exhaustiva
    cv=3,  # Usar 3 particiones en vez de 5 para acelerar la validación cruzada
    scoring='accuracy',  # Métrica para evaluar
    random_state=42,
    n_jobs=-1  # Usar todos los núcleos disponibles
)

# Entrenar el modelo con RandomizedSearchCV
random_search.fit(X_train_scaled, y_train)

# Mostrar los mejores parámetros
print("Mejores hiperparámetros encontrados:")
print(random_search.best_params_)

# Predecir con el mejor modelo en el conjunto de validación
y_pred = random_search.best_estimator_.predict(X_val_scaled)

# Evaluar el rendimiento
accuracy = accuracy_score(y_val, y_pred)
report = classification_report(y_val, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Mejores hiperparámetros encontrados:
{'kernel': 'rbf', 'gamma': 'scale', 'degree': 5, 'class_weight': None, 'C': 0.1}
Accuracy: 0.4332493702770781
Classification Report:
              precision    recall  f1-score   support

           0       0.41      0.40      0.41       159
           1       0.45      0.64      0.53       169
           2       0.00      0.00      0.00        69

    accuracy                           0.43       397
   macro avg       0.29      0.35      0.31       397
weighted avg       0.36      0.43      0.39       397



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
